## Replicar Lite 
This notebook contains a  curated list of small examples designed to show all of the components available, how they work with each other and their configuration capabilities. 

Lets load up all of the modules first here are the components available
1) __common_types__<br>
   This module contains commonly used types across the system<br>
   a) TrafficVehicle -> for returning state of traffic and ego <br> 
   b) OpendriveRoadInfo -> for input of opendrive lane information <br>
   <br>
2) __tools__<br>
   This module contains<br> 
   a) Visuaizer<br><br>
3) __traffic_config__<br>
   This module contains everything related to traffic configuration
   the most important are <br>
   a) TrafficConfigManager <br>
   b) InputRoadUserProfile<br>
   c) InputDriverProfile<br><br>
4) __traffic__ <br>
   This module contains components related to traffic simulation<br>
   a) Traffic<br>
   b) ControllableEgo<br>
   c) ControllerScheduler<br>

In [ ]:
import common_types
import tools
import traffic_config
import traffic

### Running loop 
Let's define a simple running with the following components<br>
1) Traffic<br>
2) ControllableEgo<br>
3) ControllerScheduler<br>
4) Visualizer <br> <br>

This loop will receive a _TrafficConfigManager_ object to run with a specific configuration.<br>
Some configurations we are doing here which are not part of _TrafficConfigManager_<br>
a) ControllerScheduler is configured to stay green for 20 seconds<br>
b) The visualizer is being updated at 25 fps<br>

In [ ]:
def run_traffic(config_manager, with_external_decision=False):

    # Initialization of all required objects

    ego = traffic.ControllableEgo(config_manager,
                                  with_external_decision)
    traffic_lib = traffic.Traffic(config_manager)
    visualizer = tools.Visualizer(config_manager, False)
    # Here we want each traffic light to stay green for 20 seconds
    controller_scheduler = traffic.ControllerScheduler(config_manager,
            20)

    vehicles = []

    is_running = True
    is_paused = False

    update_visualizer = True
    update_visualizer_fps_frame_limit = 25
    update_visualizer_frame_counter = 0

    while is_running:
        is_paused = visualizer.is_paused()
        if not is_paused:
            # Set to keep decision if we want to control decision externally

            if with_external_decision is True:
                ego.set_decision(traffic.Decision.KEEP)

            [light_states_simulation, light_states_visualizer] = \
                controller_scheduler.update()
            ego_state = ego.drive(vehicles, light_states_simulation)
            vehicles = traffic_lib.sync([ego_state],
                    light_states_simulation)
            vehicles.append(ego_state)

        # Update visualzer at 25 FPS

        if update_visualizer:
            # smoothed fps calculation can be sent here to visualizer
            # this will give accurate calculation RT factor
            is_running = visualizer.update(config_manager.fps,
                    vehicles, light_states_visualizer)
            update_visualizer = False

        if update_visualizer_frame_counter \
            == update_visualizer_fps_frame_limit:
            update_visualizer = True
            update_visualizer_frame_counter = 0

        update_visualizer_frame_counter += 1

# Default Configuration

Let's start simulating some traffic <br>
We are only required to give variable known as map_path for initializing <br>
TrafficConfigManager object <br><br>
All others can take on default values <br>
Some of the highlights of the configuration are <br>
a) fps = 100 <br>
b) traffic_amount = 100 <br>
c) spanwer_strategy = SPAWN_EVERYWHERE <br>
d) ego_sensor_range = 200 <br>
e) roaduser_profiles = [[Car, 0.0], [Car, 0.85], [Bus, 0.03], [Truck, 0.01], [Motorcycle, 0.02]]<br>

In [ ]:
map_file_path = \
    'maps/A10-IN-1-19KM_HW_AC_DE_BER_RELEASE_20210510.xodr'
    
config_manager = \
    traffic_config.TrafficConfigManager(map_path=map_file_path)
    
run_traffic(config_manager)

## Rate Based Spanwer

 Rates based spawner is useful for replicating real world
 traffic flows.<br>
 The use case is that we want to observe how a certain flow of  traffic behaves at junctions
 intersections and around traffic lights and more importantly 
 how it changes with adjustments to these things as well<br><br>
 
 For this configuration<br>
 a) We change the spawner type to SPAWN_RATES <br>
 b) We add a lane to spawn_point in opendrive format (road_id, lane_id, lane_section) <br>
    and rate in vehicles per hour <br>
 c) We change the traffic amount to 500 <br>

In [ ]:
config_manager.spawner_strategy = \
    traffic_config.SpawnerStrategy.SPAWN_RATES

odr_lane = common_types.OpenDriveRoadInfo(road_id=268,
        lane_id=-2, lane_section=0)
config_manager.spawn_points = [[odr_lane, 1800]]

config_manager.traffic_amount = 500

run_traffic(config_manager)

## Spawn Around Ego Vision

For this configuration <br>
a) Change spawner strategy to SPAWN_AROUND_EGO_VISION <br>
b) Make a deep copy of the current configuration<br>
c) Change sensor range to 50 meters in copied configuration<br>
d) Change traffic amount to 50 in copied configuration<br><br>

Press __S__ to visualize the sensor range in visualizer<br>

In [ ]:
config_manager.spawner_strategy = \
    traffic_config.SpawnerStrategy.SPAWN_AROUND_EGO_VISION

config_manager_copy = config_manager.copy()
config_manager_copy.ego_sensor_range = 50
config_manager_copy.traffic_amount = 50

run_traffic(config_manager_copy)

## RoadUserProfile

A recap of the default road user profile configuration <br>
The profiles themselves have a lot configuration parameters as well <br>
but we will describe the type and it's percentage here <br>
in order to keep things simple. <br>
(Car, 0.0) -> Ego Profile, The first profile must always be ego <br>
(Car, 0.85) -> Car Profile with percentage 0.85 <br>
(Bus, 0.03) -> Bus Profile with percentage 0.03 <br>
(Truck, 0.1) -> Truck Profile with percentage 0.1 <br>
(Motrocycle, 0.02) -> Motorcycle Profile with percentage 0.02 <br>
<br>    
The configuration described above defines a simulation in which <br>
we will roughly observe the above percentages of vehicle types <br>
as seen.<br>
<br>
__Note__: <br>
We can have multiple profiles of the same type and any other <br>
mix possible. The only thing we need to take care of is that <br> 
percentage other than ego should sum to 1.0 <br>

Let's create a traffic simulation with only buses <br><br>
__InputRoadUserProfile__ is the object used to define a road user profile <br>
If you know the parameters you can construct the whole object to your <br>
liking however if you don't have accurate values for a given profile <br><br>
We have given the option to specify the type of profile that you want <br>
We will construct the complete road user profile just from the __RoadUserType__ <br>

Let's create two bus profiles <br>
a) The default bus profile <br>
b) The default bus profile with length changed to 15 meters <br>

In [ ]:
roaduser_bus = \
    traffic_config.InputRoadUserProfile(traffic_config.RoadUserType.BUS)
    
roaduser_bus_long = \
    traffic_config.InputRoadUserProfile(traffic_config.RoadUserType.BUS)
    
# Let's change the length of the bus to 15 meters instead of the default
roaduser_bus_long.length = 15

Get the roaduser_profiles which are already present in <br>
__TrafficConfigManager__ object and take the ego profile from that<br>
<br>
The first object from the list and than again 0 object as the objects of list are of type <br>
Tuple[InputRoadUserProfile, Percentage] <br>

In [ ]:
ego_profile = config_manager.roaduser_profiles[0][0]

 Now construct the road user profiles with both bus profiles

In [ ]:
config_manager.roaduser_profiles = [[ego_profile, 0.0],
            [roaduser_bus, 0.5], [roaduser_bus_long, 0.5]]

run_traffic(config_manager)

## DriverProfile

Each vehicle profile has associated driver profiles as well <br>
By default only one driver profile is pushed with 1.0 percentage <br>
<br>
More can be pushed as well if needed if user wants same vehicle profile <br>
with different types of driver driving them<br>
<br>
Let's change the default moderate profile to an aggressive one <br>
So we will see more aggressive values for each of the road users <br>
In this configuration <br>
<br>
We will set all vehicle profiles to cars <br>

In [ ]:
roaduser_car = \
    traffic_config.InputRoadUserProfile(traffic_config.RoadUserType.CAR)

driver_profile = roaduser_car.driver_profiles[0][0]
driver_profile.p1 = 0.0
driver_profile.p2 = 0.0
driver_profile.p3 = 0.0
driver_profile.p4 = 1.0
driver_profile.target_speed = 40
roaduser_car.driver_profiles = [[driver_profile, 1.0]]

config_manager.roaduser_profiles = [[ego_profile, 0.0],
        [roaduser_car, 1.0]]

run_traffic(config_manager)

## Externally Controlled Decision

We will run the ego with externally controlled decision<br>
We will enforce it keep going in the current lane <br>
<br>
Other decisions are also available like left and right<br>
If the decision is not possible it will be ignored.<br>
and the result will be returnd in the set_decision() function. <br>
<br>
This is acheived by passing True as second <br>
parameter and than handled inside run_traffic() function<br>
with the ControllableEgo object <br>

In [ ]:
run_traffic(config_manager, True)

## Urban Traffic with Traffic Light Signals

In this configuration <br>
a) Change map to an urban one i.e with intersections and traffic lights as well <br>
b) roaduser_profiles to cars and pedestrians only <br>
c) spawner_strategy to SPAWN_EVERYWHERE <br>
d) traffic_amount to 400 to fill out the map <br>

In [ ]:
config_manager.map_path = \
    'maps/BMW-RA+JT-Headquater-11KM_UR_AC_DE_MUN_RELEASE_20210901.xodr'

roaduser_pedestrian = \
    traffic_config.InputRoadUserProfile(
                    traffic_config.RoadUserType.PEDESTRIAN)

config_manager.roaduser_profiles = [[ego_profile, 0.0],
        [roaduser_car, 0.4], [roaduser_pedestrian, 0.6]]

config_manager.spawner_strategy = \
    traffic_config.SpawnerStrategy.SPAWN_EVERYWHERE

config_manager.traffic_amount = 400

run_traffic(config_manager)